In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import ast


In [191]:
df=pd.read_csv("flipkart_com-ecommerce_sample.csv")
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [192]:
print(df.shape)
df.info()

(20000, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_sp

In [193]:
df.isnull().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

In [194]:
df['retail_price'] = df['retail_price'].fillna(df['retail_price'].median())
df['discounted_price'] = df['discounted_price'].fillna(df['discounted_price'].median())
df['image'] = df['image'].fillna('No Image')
df['description'] = df['description'].fillna('No Description')
df['brand'] = df['brand'].fillna('Unknown Brand')
df['product_specifications'] = df['product_specifications'].fillna('{}')

In [195]:
df.isnull().sum()

uniq_id                    0
crawl_timestamp            0
product_url                0
product_name               0
product_category_tree      0
pid                        0
retail_price               0
discounted_price           0
image                      0
is_FK_Advantage_product    0
description                0
product_rating             0
overall_rating             0
brand                      0
product_specifications     0
dtype: int64

In [196]:
def extract_main_category(tree_str):
    try:
        # Parse the string as list and get first element
        if isinstance(tree_str, str):
            # Remove brackets and quotes, then split
            categories = tree_str.strip('[]').split(' >> ')
            if categories and categories[0]:
                return categories[0].replace('"', '').strip()
        return 'unknown'
    except:
        return 'unknown'
df['main_category'] = df['product_category_tree'].apply(extract_main_category)

In [197]:
category_counts = df['main_category'].value_counts()
print(f"\nNumber of unique categories: {len(category_counts)}")
print("\nCategory distribution:")
print(category_counts.head(20))


Number of unique categories: 265

Category distribution:
main_category
Clothing                      6198
Jewellery                     3531
Footwear                      1227
Mobiles & Accessories         1099
Automotive                    1012
Home Decor & Festive Needs     929
Beauty and Personal Care       710
Home Furnishing                700
Kitchen & Dining               647
Computers                      578
Watches                        530
Baby Care                      483
Tools & Hardware               391
Toys & School Supplies         330
Pens & Stationery              313
Bags, Wallets & Belts          265
Furniture                      180
Sports & Fitness               166
Cameras & Accessories           82
Home Improvement                81
Name: count, dtype: int64


In [ ]:
MIN_SAMPLES_PER_CATEGORY = 165
category_counts_filtered = category_counts[category_counts >= MIN_SAMPLES_PER_CATEGORY]
valid_categories = set(category_counts_filtered.index)

print(f"Categories before filtering: {len(category_counts)}")
print(f"Categories after filtering: {len(category_counts_filtered)}")

df_filtered = df[df['main_category'].isin(valid_categories)].copy()
print(f"Dataset size after filtering: {df_filtered.shape}")


Filtering categories with at least 165 samples...
Categories before filtering: 265
Categories after filtering: 18
Dataset size after filtering: (19289, 16)


In [ ]:
# combining product features
def combine_features(row):
    """Combine product name, description, and brand into one text feature"""
    name = str(row['product_name'])
    desc = str(row['description'])
    brand = str(row['brand'])
    
    specs = str(row['product_specifications'])
    try:
        if specs and specs != 'nan':
            specs_dict = json.loads(specs.replace('=>', ':'))
            specs_text = ' '.join([f"{item['key']} {item.get('value', '')}" 
                                 for item in specs_dict.get('product_specification', []) 
                                 if isinstance(item, dict)])
        else:
            specs_text = ''
    except:
        specs_text = ''
    
    combined_text = f"{name} {desc} {brand} {specs_text}"
    combined_text = re.sub(r'[^\w\s]', ' ', combined_text)
    combined_text = re.sub(r'\s+', ' ', combined_text).strip().lower()
    
    return combined_text

df_filtered['combined_text'] = df_filtered.apply(combine_features, axis=1)


In [200]:
empty_text_mask = df_filtered['combined_text'].str.strip() == ''
print(f"Empty texts found: {empty_text_mask.sum()}")
df_filtered = df_filtered[~empty_text_mask]

print(f"Final dataset size: {df_filtered.shape}")


Empty texts found: 0
Final dataset size: (19289, 17)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [202]:
X = df_filtered['combined_text']
y = df_filtered['main_category']

In [203]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Number of classes: {len(label_encoder.classes_)}")
print("Classes:", label_encoder.classes_)

Number of classes: 18
Classes: ['Automotive' 'Baby Care' 'Bags, Wallets & Belts'
 'Beauty and Personal Care' 'Clothing' 'Computers' 'Footwear' 'Furniture'
 'Home Decor & Festive Needs' 'Home Furnishing' 'Jewellery'
 'Kitchen & Dining' 'Mobiles & Accessories' 'Pens & Stationery'
 'Sports & Fitness' 'Tools & Hardware' 'Toys & School Supplies' 'Watches']


In [204]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8
)

X_vectorized = tfidf_vectorizer.fit_transform(X)
print(f"Vectorized features shape: {X_vectorized.shape}")

Vectorized features shape: (19289, 5000)


In [205]:
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, 
    y_encoded, 
    test_size=0.2, 
    random_state=42,
    stratify=y_encoded)

In [206]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 15431
Test set size: 3858


In [207]:
model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced',
    C=1.0
)

In [208]:
calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv=3)
calibrated_model.fit(X_train, y_train)


CalibratedClassifierCV(cv=3,
                       estimator=LogisticRegression(class_weight='balanced',
                                                    max_iter=1000,
                                                    random_state=42))

In [209]:
y_pred = calibrated_model.predict(X_test)
y_pred_proba = calibrated_model.predict_proba(X_test)

In [210]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9733

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       202
           1       0.94      0.74      0.83        97
           2       0.92      0.92      0.92        53
           3       0.95      0.99      0.97       142
           4       0.98      1.00      0.99      1240
           5       0.98      0.95      0.96       116
           6       0.99      0.99      0.99       245
           7       0.97      1.00      0.99        36
           8       0.95      0.97      0.96       186
           9       0.95      0.99      0.97       140
          10       1.00      0.99      1.00       706
          11       0.93      0.98      0.95       129
          12       0.98      0.99      0.98       220
          13       0.91      0.68      0.78        63
          14       0.82      0.97      0.89        33
          15       1.00      0.96      0.98        78
          16       0.82      0.80      0

In [211]:
joblib.dump(calibrated_model, 'product_category_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']

In [212]:
category_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
with open('category_mapping.json', 'w') as f:
    json.dump(category_mapping, f, indent=2)

print("Model training completed successfully!")

Model training completed successfully!


In [ ]:
def predict_category(product_name, description, brand, product_specifications=""):
    """Function to predict category for new product"""
    # Combine features
    combined_text = combine_features({
        'product_name': product_name,
        'description': description,
        'brand': brand,
        'product_specifications': product_specifications
    })
    text_vectorized = tfidf_vectorizer.transform([combined_text])
    
    # Predict
    probabilities = calibrated_model.predict_proba(text_vectorized)[0]
    predicted_class_idx = np.argmax(probabilities)
    confidence = probabilities[predicted_class_idx]
    predicted_category = label_encoder.inverse_transform([predicted_class_idx])[0]
    # Top 3 Prediction
    top_3_indices = np.argsort(probabilities)[-3:][::-1]
    top_3_categories = label_encoder.inverse_transform(top_3_indices)
    top_3_confidences = probabilities[top_3_indices]
    
    return {
        'predicted_category': predicted_category,
        'confidence': float(confidence),
        'top_3_predictions': [
            {'category': cat, 'confidence': float(conf)} 
            for cat, conf in zip(top_3_categories, top_3_confidences)
        ]
    }


In [214]:
print("\nTesting prediction with sample data...")
sample_prediction = predict_category(
    product_name="Nike Running Shoes",
    description="Comfortable running shoes with air cushioning",
    brand="Nike"
)
print("Sample prediction:")
print(sample_prediction)


Testing prediction with sample data...
Sample prediction:
{'predicted_category': 'Footwear', 'confidence': 0.91749466409034, 'top_3_predictions': [{'category': 'Footwear', 'confidence': 0.91749466409034}, {'category': 'Clothing', 'confidence': 0.03693161643864153}, {'category': 'Automotive', 'confidence': 0.011432919052622332}]}
